In [29]:
import numpy as np
import re
from functools import reduce
import preprocessing as preproc
import time
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
import helpers
import models
import word2vec as w2v

In [ ]:
start_time = time.time()

In [ ]:
preproc.do_preprocessing('../twitter-datasets/train_pos_full.txt')
preproc.do_preprocessing('../twitter-datasets/train_neg_full.txt')

In [ ]:
lines, y = preproc.return_processed_trainset_and_y(False)

In [ ]:
print(len(lines))
lines[:5]

In [ ]:
vectorizer = TfidfVectorizer(stop_words=None, ngram_range=(1,1), sublinear_tf=True, max_features=None)
X = vectorizer.fit_transform(lines)
X = preproc.standardize_with_standrard_scaler(X)

In [ ]:
clf = models.return_model('SVM')

In [ ]:
# rs = ShuffleSplit(n_splits=4, test_size=.25, random_state=0)
kf = KFold(n_splits=4, shuffle=True, random_state=0)

scores = cross_val_score(clf, X, y, cv=kf)
print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# clf = LogisticRegression().fit(X, y)
forest = RandomForestClassifier(n_estimators=50, max_depth=50, random_state=4, n_jobs=-1)

kf = KFold(n_splits=4, shuffle=True, random_state=0)
scores = cross_val_score(forest, X, y, cv=kf)
print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

In [ ]:
preproc.do_preprocessing('../twitter-datasets/test_data.txt')

In [ ]:
with open('../twitter-datasets/test_data_processed.txt') as test_in:
    test_lines = test_in.readlines()

In [ ]:
print(len(test_lines))
print(test_lines[:5])

In [ ]:
X_test = vectorizer.transform(test_lines)

y_pred = clf.predict(X_test)

helpers.create_submission_csv(y_pred)

In [ ]:
print(np.unique(y_pred))

In [ ]:
elapsed_time = divmod(round((time.time() - start_time)), 60)
print('------\nElapsed time: {m} min {s} sec\n'.format(m=elapsed_time[0], s=elapsed_time[1]))

BELOW: testing...

In [2]:
lines, y = preproc.return_processed_trainset_and_y(False)
len(lines)

200000

In [3]:
lines1 = lines[:50000] + lines[100000:150000]
y1 = np.concatenate((y[:50000], y[100000:150000]), axis=0)

In [4]:
len(lines1)

100000

In [5]:
len(y1)

100000

In [48]:
vectorizer = TfidfVectorizer(stop_words=None, ngram_range=(1,2), sublinear_tf=True, max_features=2000
)
X = vectorizer.fit_transform(lines1)

In [49]:
X.shape

(100000, 2000)

In [50]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Conv1D, Embedding, GlobalMaxPooling1D

In [51]:
X

<100000x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 1166294 stored elements in Compressed Sparse Row format>

In [57]:
# create model
model = Sequential()
model.add(Embedding(X.shape[1], 1000, input_length=2000))
model.add(Conv1D(filters=1, kernel_size=2, padding='valid', activation='relu', strides=1))
# model.add(Dense(units=12, input_dim=X.shape[1], activation='relu'))
# model.add(Dense(8, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
model.add(GlobalMaxPooling1D())
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 2000, 1000)        2000000   
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 1999, 1)           2001      
_________________________________________________________________
global_max_pooling1d_10 (Glo (None, 1)                 0         
Total params: 2,002,001
Trainable params: 2,002,001
Non-trainable params: 0
_________________________________________________________________


In [58]:
len(lines1)

100000

In [59]:
#compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [60]:
X.shape

(100000, 2000)

In [61]:
#fit the model
model.fit(x=X, y=y1, batch_size=32, epochs=1)

Epoch 1/1
 24832/100000 [======>.......................] - ETA: 21:02 - loss: -0.0303 - acc: 0.0000e+00

KeyboardInterrupt: 

In [28]:
scores = model.evaluate(X, y1)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

100000/100000 [==============================] - 113s 1ms/step

acc: 0.00%
